In [1]:
# import random
# import heapq
# from collections import deque

# class BlocksWorldState:
#     def __init__(self, state):
#         self.state = state

#     def __lt__(self, other):
#         return False  # Arbitrary ordering for heapq

#     def __eq__(self, other):
#         return self.state == other.state

#     def __hash__(self):
#         return hash(tuple(self.state))

#     def is_goal(self, goal_state):
#         return self.state == goal_state

#     def get_successors(self):
#         successors = []
#         for i in range(1, len(self.state)):
#             for j in range(len(self.state)):
#                 if j != i and j != self.state[i]:
#                     new_state = self.state.copy()
#                     new_state[i] = j
#                     successors.append(BlocksWorldState(new_state))
#         return successors

#     def heuristic(self, goal_state):
#         return sum(1 for i in range(len(self.state)) if self.state[i] != goal_state[i])

# def generate_random_state(num_blocks):
#     state = [0] * num_blocks
#     for i in range(1, num_blocks):
#         state[i] = random.randint(0, i - 1)
#     return state

# def random_walk(initial_state, num_steps):
#     state = initial_state.copy()
#     for _ in range(num_steps):
#         block = random.randint(1, len(state) - 1)
#         new_position = random.randint(0, len(state) - 1)
#         if new_position != block and new_position != state[block]:
#             state[block] = new_position
#     return state

# def a_star_search(initial_state, goal_state):
#     start_node = BlocksWorldState(initial_state)
#     frontier = [(start_node.heuristic(goal_state), 0, start_node)]
#     came_from = {}
#     cost_so_far = {start_node: 0}
    
#     while frontier:
#         _, current_cost, current_node = heapq.heappop(frontier)
        
#         if current_node.is_goal(goal_state):
#             return reconstruct_path(came_from, start_node, current_node), cost_so_far
        
#         for successor in current_node.get_successors():
#             new_cost = current_cost + 1
#             if successor not in cost_so_far or new_cost < cost_so_far[successor]:
#                 cost_so_far[successor] = new_cost
#                 priority = new_cost + successor.heuristic(goal_state)
#                 heapq.heappush(frontier, (priority, new_cost, successor))
#                 came_from[successor] = current_node
    
#     return None, cost_so_far  # No solution found

# def reconstruct_path(came_from, start, goal):
#     path = deque([goal])
#     current = goal
#     while current != start:
#         current = came_from[current]
#         path.appendleft(current)
#     return list(path)

# def save_search_tree(cost_so_far, filename):
#     with open(filename, 'w') as f:
#         for state, cost in cost_so_far.items():
#             f.write(f"State: {state.state}, Cost: {cost}\n")

# def main():
#     num_instances = 5
#     num_blocks = 5
#     num_steps = 10  # Number of steps for random walk
    
#     for i in range(num_instances):
#         initial_state = generate_random_state(num_blocks)
#         goal_state = random_walk(initial_state, num_steps)
        
#         print(f"Instance {i+1}:")
#         print(f"Initial state: {initial_state}")
#         print(f"Goal state: {goal_state}")
        
#         path, cost_so_far = a_star_search(initial_state, goal_state)
        
#         if path:
#             print(f"Solution found with {len(path)-1} steps:")
#             for state in path:
#                 print(state.state)
#         else:
#             print("No solution found.")
        
#         save_search_tree(cost_so_far, f"search_tree_instance_{i+1}.txt")
#         print(f"Search tree saved to search_tree_instance_{i+1}.txt")
#         print()

# main()

Instance 1:
Initial state: [0, 0, 1, 0, 3]
Goal state: [0, 3, 1, 1, 3]
Solution found with 2 steps:
[0, 0, 1, 0, 3]
[0, 3, 1, 0, 3]
[0, 3, 1, 1, 3]
Search tree saved to search_tree_instance_1.txt

Instance 2:
Initial state: [0, 0, 0, 0, 2]
Goal state: [0, 0, 1, 1, 0]
Solution found with 3 steps:
[0, 0, 0, 0, 2]
[0, 0, 1, 0, 2]
[0, 0, 1, 1, 2]
[0, 0, 1, 1, 0]
Search tree saved to search_tree_instance_2.txt

Instance 3:
Initial state: [0, 0, 1, 2, 1]
Goal state: [0, 3, 3, 1, 0]
Solution found with 4 steps:
[0, 0, 1, 2, 1]
[0, 3, 1, 2, 1]
[0, 3, 1, 2, 0]
[0, 3, 1, 1, 0]
[0, 3, 3, 1, 0]
Search tree saved to search_tree_instance_3.txt

Instance 4:
Initial state: [0, 0, 0, 1, 0]
Goal state: [0, 0, 0, 2, 2]
Solution found with 2 steps:
[0, 0, 0, 1, 0]
[0, 0, 0, 2, 0]
[0, 0, 0, 2, 2]
Search tree saved to search_tree_instance_4.txt

Instance 5:
Initial state: [0, 0, 0, 1, 2]
Goal state: [0, 2, 3, 0, 0]
Solution found with 4 steps:
[0, 0, 0, 1, 2]
[0, 0, 3, 1, 2]
[0, 0, 3, 1, 0]
[0, 0, 3, 0, 0]


In [6]:
import random
import heapq
import argparse
from typing import List, Tuple, Dict, Optional

class Block:
    def __init__(self, name: str):
        self.name = name

class BlocksworldState:
    def __init__(self, stacks: List[List[Block]]):
        self.stacks = stacks

    def __str__(self):
        string = ""
        for stack in self.stacks:
            string += "("
            string += " ".join(block.name for block in stack)
            string += ")"
        return string

    def __eq__(self, other):
        return str(self) == str(other)

    def __hash__(self):
        return hash(str(self))

def generate_initial_state(num_blocks: int, max_stacks: int) -> BlocksworldState:
    blocks = [Block(chr(65 + i)) for i in range(num_blocks)]
    stacks = [[] for _ in range(random.randint(1, max_stacks))]
    for block in blocks:
        random.choice(stacks).append(block)
    return BlocksworldState([stack for stack in stacks])

def random_walk(initial_state: BlocksworldState, num_steps: int) -> BlocksworldState:
    current_state = initial_state
    for _ in range(num_steps):
        actions = get_valid_actions(current_state)
        if not actions:
            break
        action = random.choice(actions)
        current_state = apply_action(current_state, action)
    return current_state

def get_valid_actions(state: BlocksworldState) -> List[Tuple[int, int]]:
    actions = []
    for i, stack in enumerate(state.stacks):
        if stack:
            for j in range(len(state.stacks)):
                if i != j:
                    actions.append((i, j))
    return actions

def apply_action(state: BlocksworldState, action: Tuple[int, int]) -> BlocksworldState:
    from_stack, to_stack = action
    new_stacks = [stack[:] for stack in state.stacks]
    block = new_stacks[from_stack].pop()
    new_stacks[to_stack].append(block)
    return BlocksworldState([stack for stack in new_stacks if stack])

class Node:
    def __init__(self, state: BlocksworldState, g: int, h: int, parent: Optional['Node'] = None, action: Optional[Tuple[int, int]] = None):
        self.state = state
        self.g = g
        self.h = h
        self.f = g + h
        self.parent = parent
        self.action = action
        self.height = parent.height + 1 if parent else 0

    def __lt__(self, other):
        return self.f < other.f

def heuristic(state: BlocksworldState, goal: BlocksworldState) -> int:
    return sum(1 for stack1, stack2 in zip(state.stacks, goal.stacks) if stack1 != stack2)

def astar(initial: BlocksworldState, goal: BlocksworldState) -> Tuple[Optional[Node], Dict[BlocksworldState, Node]]:
    start_node = Node(initial, 0, heuristic(initial, goal))
    open_list = [start_node]
    closed_set = set()
    node_map = {initial: start_node}

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.state == goal:
            return current_node, node_map

        closed_set.add(current_node.state)

        for action in get_valid_actions(current_node.state):
            child_state = apply_action(current_node.state, action)
            if child_state in closed_set:
                continue

            g = current_node.g + 1
            h = heuristic(child_state, goal)
            child_node = Node(child_state, g, h, current_node, action)

            if child_state not in node_map or g < node_map[child_state].g:
                node_map[child_state] = child_node
                heapq.heappush(open_list, child_node)

    return None, node_map

def main():
    # parser = argparse.ArgumentParser(description="Blocksworld A* Planner")
    # parser.add_argument("--num-blocks", type=int, default=5, help="Number of blocks")
    # parser.add_argument("--max-stacks", type=int, default=3, help="Maximum number of stacks")
    # parser.add_argument("--num-steps", type=int, default=10, help="Number of steps for random walk")
    # args = parser.parse_args()
    num_blocks = 5
    max_stacks = 4
    num_steps = 10

    initial_state = generate_initial_state(num_blocks=num_blocks, max_stacks=max_stacks)
    goal_state = random_walk(initial_state, num_steps=num_steps)

    print(f"Initial state: {initial_state}")
    print(f"Goal state: {goal_state}")

    solution_node, search_tree = astar(initial_state, goal_state)

    if solution_node:
        print(f"Solution found with {solution_node.g} steps:")
        path = []
        current = solution_node
        while current:
            path.append(current)
            current = current.parent
        for node in reversed(path):
            print(f"Step {node.g}: {node.state}")
    else:
        print("No solution found.")

    print(f"\nSearch tree statistics:")
    print(f"Total nodes explored: {len(search_tree)}")
    print(f"Maximum tree height: {max(node.height for node in search_tree.values())}")

main()

Initial state: (B D)(A)(C E)
Goal state: (A B)(C E D)
Solution found with 2 steps:
Step 0: (B D)(A)(C E)
Step 1: (B)(A)(C E D)
Step 2: (A B)(C E D)

Search tree statistics:
Total nodes explored: 20
Maximum tree height: 2
